In [1]:
import subprocess as sp
import sys,os,shutil,pickle,time
from os.path import join
from os.path import expanduser


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('iphone_battery_review.csv')
df.head(5)

,stars,comment
0,5,"In my opinion, don’t think much about the pho..."
1,5,"In my opinion, don’t think much about the pho..."
2,4,My mom got this phone after using oneplus 5t f...
3,4,Been almost 10 months I have been using the iP...
4,5,Pros:Camera quality is 09/10.Display 09/10Batt...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stars    650 non-null    int64 
 1   comment  650 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.3+ KB


In [5]:
df.describe().round(1)

,stars
count,650.0
mean,4.3
std,0.6
min,3.0
25%,4.0
50%,4.0
75%,5.0
max,5.0


In [6]:
# Determine how many missing values exist in the collection, in which case you can use .sum() chained onto is.na()
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0


In [7]:
df= df.dropna()
df.shape

(650, 2)

In [8]:
#pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
df['stopwords'] = df['comment'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['comment','stopwords']].head()

,comment,stopwords
0,"In my opinion, don’t think much about the pho...",40
1,"In my opinion, don’t think much about the pho...",40
2,My mom got this phone after using oneplus 5t f...,158
3,Been almost 10 months I have been using the iP...,282
4,Pros:Camera quality is 09/10.Display 09/10Batt...,9


In [10]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['comment'].apply(lambda x: count_punct(x))

In [11]:
df[['comment','punctuation']].head()

,comment,punctuation
0,"In my opinion, don’t think much about the pho...",31
1,"In my opinion, don’t think much about the pho...",31
2,My mom got this phone after using oneplus 5t f...,48
3,Been almost 10 months I have been using the iP...,92
4,Pros:Camera quality is 09/10.Display 09/10Batt...,12


In [12]:
df['hastags'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df[['comment','hastags']].head()

,comment,hastags
0,"In my opinion, don’t think much about the pho...",0
1,"In my opinion, don’t think much about the pho...",0
2,My mom got this phone after using oneplus 5t f...,0
3,Been almost 10 months I have been using the iP...,0
4,Pros:Camera quality is 09/10.Display 09/10Batt...,0


In [13]:
df.hastags.loc[df.hastags != 0].count()

0

In [14]:
df['numerics'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['comment','numerics']].head()

,comment,numerics
0,"In my opinion, don’t think much about the pho...",1
1,"In my opinion, don’t think much about the pho...",1
2,My mom got this phone after using oneplus 5t f...,1
3,Been almost 10 months I have been using the iP...,9
4,Pros:Camera quality is 09/10.Display 09/10Batt...,1


In [15]:
df['upper'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['comment','upper']].head()

,comment,upper
0,"In my opinion, don’t think much about the pho...",6
1,"In my opinion, don’t think much about the pho...",6
2,My mom got this phone after using oneplus 5t f...,9
3,Been almost 10 months I have been using the iP...,33
4,Pros:Camera quality is 09/10.Display 09/10Batt...,2


In [16]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['comment'].head()

0    in my opinion, don’t think much about the phon...
1    in my opinion, don’t think much about the phon...
2    my mom got this phone after using oneplus 5t f...
3    been almost 10 months i have been using the ip...
4    pros:camera quality is 09/10.display 09/10batt...
Name: comment, dtype: object

In [17]:
df['comment'] = df['comment'].str.replace('[^\w\s]','')
df['comment'].head()

<ipython-input-17-66a30b7fedcd>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comment'] = df['comment'].str.replace('[^\w\s]','')


0    in my opinion dont think much about the phone ...
1    in my opinion dont think much about the phone ...
2    my mom got this phone after using oneplus 5t f...
3    been almost 10 months i have been using the ip...
4    proscamera quality is 0910display 0910battery ...
Name: comment, dtype: object

In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['comment'].sample(10)

408    moved phone samsung s8 transition ios easy how...
492    almost 10 months using iphone 11 128 gb apple ...
331    mom got phone using oneplus 5t 2 years change ...
529    first time new iphone user decades android own...
557    battery lifecant complain much pretty decent 5...
76     awesome phone specially youre upgrading older ...
328    moved phone samsung s8 transition ios easy how...
115    rating right nowbut want make one thing clear ...
32     mom got phone using oneplus 5t 2 years change ...
532    almost 10 months using iphone 11 128 gb apple ...
Name: comment, dtype: object

In [19]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

In [20]:
# remove all urls from df
import re
import string

df['comment'] = df['comment'].apply(lambda x: remove_url(x))

In [21]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [22]:
# remove all html tags from df
df['comment'] = df['comment'].apply(lambda x: remove_html(x))

In [23]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [24]:
#Example
remove_emoji("Omg another Earthquake 😔😔")

'Omg another Earthquake '

In [25]:
# remove all emojis from df
df['comment'] = df['comment'].apply(lambda x: remove_emoji(x))

In [26]:
!pip install emot

Defaulting to user installation because normal site-packages is not writeable


In [27]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [28]:
#Example
remove_emoticons("Hello :-)")

'Hello '

In [29]:
df['comment'] = df['comment'].apply(lambda x: remove_emoticons(x))

In [30]:
from textblob import TextBlob
df['comment'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    opinion dont think much phone get attaching ni...
1    opinion dont think much phone get attaching ni...
2    mon got phone using onerous it 2 years change ...
3    almost 10 months using phone 11 128 go apple k...
4    proscamera quality 0910display 0910battery lif...
Name: comment, dtype: object

In [31]:
# We could do some of the cleaning steps as a sum of opreation like this:

# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [32]:
df['comment'] = df.comment.apply(round1)
df.comment

0      opinion dont think much phone get attaching ni...
1      opinion dont think much phone get attaching ni...
2      mom got phone using oneplus   years change wan...
3      almost  months using iphone   gb apple known o...
4      proscamera quality   life  issue face heating ...
                             ...                        
645    rating right nowbut want make one thing clear ...
646    awesome phone specially youre upgrading older ...
647    battery lifecant complain much pretty decent  ...
648    moved phone samsung  transition ios easy howev...
649    first time new iphone user decades android own...
Name: comment, Length: 650, dtype: object

In [33]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [34]:
df['comment'] = df.comment.apply(round2)
df.comment

0      opinion dont think much phone get attaching ni...
1      opinion dont think much phone get attaching ni...
2      mom got phone using oneplus   years change wan...
3      almost  months using iphone   gb apple known o...
4      proscamera quality   life  issue face heating ...
                             ...                        
645    rating right nowbut want make one thing clear ...
646    awesome phone specially youre upgrading older ...
647    battery lifecant complain much pretty decent  ...
648    moved phone samsung  transition ios easy howev...
649    first time new iphone user decades android own...
Name: comment, Length: 650, dtype: object

In [35]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:20]
freq

phone      2340
iphone     2145
apple      1105
great       910
good        845
charger     780
one         715
best        715
screen      715
quality     650
get         650
life        650
day         585
camera      585
user        585
dont        585
battery     585
display     520
price       520
android     520
dtype: int64

In [36]:
df['word_count'] = df['comment'].apply(lambda x: len(str(x).split(" ")))
df[['comment','word_count']].head()

,comment,word_count
0,opinion dont think much phone get attaching ni...,77
1,opinion dont think much phone get attaching ni...,77
2,mom got phone using oneplus years change wan...,201
3,almost months using iphone gb apple known o...,382
4,proscamera quality life issue face heating ...,31


In [37]:
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0
stopwords,0,0.0
punctuation,0,0.0
hastags,0,0.0
numerics,0,0.0
upper,0,0.0
word_count,0,0.0


In [38]:
df['char_count'] = df['comment'].str.len() ## this also includes spaces
df[['comment','char_count']].head()

,comment,char_count
0,opinion dont think much phone get attaching ni...,414
1,opinion dont think much phone get attaching ni...,414
2,mom got phone using oneplus years change wan...,1384
3,almost months using iphone gb apple known o...,2635
4,proscamera quality life issue face heating ...,209


In [39]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))

In [40]:
df['avg_word'] = df['comment'].apply(lambda x: avg_word(x)).round(1)
df[['comment','avg_word']].head()

,comment,avg_word
0,opinion dont think much phone get attaching ni...,5.3
1,opinion dont think much phone get attaching ni...,5.3
2,mom got phone using oneplus years change wan...,6.0
3,almost months using iphone gb apple known o...,6.2
4,proscamera quality life issue face heating ...,6.6


In [41]:
df.sample(2)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
312,4,almost months using iphone gb apple known o...,282,92,0,9,33,382,2635,6.2
18,3,moved phone samsung transition ios easy howev...,36,12,0,0,2,42,270,5.6


In [42]:
#df['Time'] = pd.to_datetime(df['Time'],unit='s')

In [43]:
#df= df.drop('ProfileName', axis= 1)

In [44]:
list(df)

['stars',
 'comment',
 'stopwords',
 'punctuation',
 'hastags',
 'numerics',
 'upper',
 'word_count',
 'char_count',
 'avg_word']

In [45]:
df.sample(5)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
184,5,last months waiting best possible phone every...,143,31,0,7,3,178,1085,5.5
151,4,mom got phone using oneplus years change wan...,158,48,0,1,9,201,1384,6.0
366,5,awesome phone specially youre upgrading older ...,17,9,0,0,0,28,200,6.2
174,5,last months waiting best possible phone every...,143,31,0,7,3,178,1085,5.5
362,4,almost months using iphone gb apple known o...,282,92,0,9,33,382,2635,6.2


In [57]:
add_words = ["get",     
"use",     
"first",      
"like","u","goes","im"      
]

stop_words = set(stopwords.words("english"))
stop_added = stop_words.union(add_words)

In [58]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_added))
df['comment'].sample(10)

592    almost months using iphone gb apple known one ...
270    opinion dont think much phone attaching nightm...
171    mom got phone using oneplus years change wante...
412    almost months using iphone gb apple known one ...
97     battery lifecant complain much pretty decent w...
15     rating right nowbut want make one thing clear ...
241    mom got phone using oneplus years change wante...
287    battery lifecant complain much pretty decent w...
573    proscamera quality life issue face heating iss...
147    battery lifecant complain much pretty decent w...
Name: comment, dtype: object

In [59]:
df1= df

In [60]:
mask = df1.comment.str.endswith('br') 
df1.loc[mask, 'comment'] = df1.loc[mask, 'comment'].str[:-2]

In [61]:
df1['comment'] = df1['comment'].str.rstrip('tty')

In [62]:
df1['comment'].apply(lambda x: x[:-2] if x.endswith('tty') else x)

0      opinion dont think much phone attaching nightm...
1      opinion dont think much phone attaching nightm...
2      mom got phone using oneplus years change wante...
3      almost months using iphone gb apple known one ...
4      proscamera quality life issue face heating iss...
                             ...                        
645    rating right nowbut want make one thing clear ...
646    awesome phone specially youre upgrading older ...
647    battery lifecant complain much pretty decent w...
648    moved phone samsung transition ios easy howeve...
649    time new iphone user decades android ownership...
Name: comment, Length: 650, dtype: object

In [63]:
df1.loc[df1.comment.str.endswith('br'), 'comment']


Series([], Name: comment, dtype: object)

In [64]:
df1.loc[df1.punctuation >= 1000].comment.tolist()

[]

In [65]:
df.loc[df.punctuation >= 1000].comment.tolist()


[]

In [66]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:50]
freq

phone         2340
iphone        2145
apple         1105
great          910
good           845
charger        780
screen         715
best           715
one            715
quality        650
life           650
battery        585
user           585
dont           585
day            585
camera         585
display        520
even           520
price          520
also           520
android        520
decent         455
oneplus        455
face           455
calls          390
oled           390
well           390
samsung        390
usage          390
really         390
lcd            390
ios            390
got            325
want           325
etc            325
easily         325
experience     325
easy           325
notice         325
works          325
say            325
last           325
pro            325
better         260
wont           260
ecosystem      260
amazing        260
right          260
review         260
charge         260
dtype: int64

In [67]:
df.to_csv('iphonebatteryreviews2.csv', index=False)